In [ ]:
import os

# Define root directory where the simulation files are located
root_directory = "/n17data/tersenov/SLICS/Cosmo_DES"

# Directories to exclude
exclude_dirs = ["SLICS_HR", "fid_a", "fid_f"]

# Open the master text file in write mode
master_file_path = "master_file.txt"
with open(master_file_path, "w") as master_file:
    # Iterate over the files and subdirectories in the root directory
    for root, dirs, files in os.walk(root_directory):
        # Exclude the specified directories
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        
        for file_name in files:
            # Write the file path to the master file
            file_path = os.path.join(root, file_name)
            master_file.write(file_path + "\n")

In [ ]:
import numpy as np

def process_files(file_paths):
    """Reads the information in the filename.

    Parameters
    ----------
    file_paths : list of str (.txt file)
        List containing the paths to the files to be processed

    Returns
    -------
    data : recarray
        Numpy recarray containing the information extracted from the file names

    """

    # Make empty recarray to store the data
    data = np.recarray(len(file_paths), dtype=[('id', int), ('seed', 'U1'), ('bin', int), ('LOS', int), ('tile', int)])

    # Iterate over the file paths and process each file
    for i, file_path in enumerate(file_paths):
        # Extract the file name from the file path
        file_name = file_path.split("/")[-1]
        
        # Split file name into parts
        file_parts = file_name.split("_")
        
        id = int(file_parts[2])
        seed = file_parts[3]
        unknown_number = int(file_parts[4])
        bin = int(file_parts[5][3:])  # Extract the number after "Bin"
        LOS = int(file_parts[6][3:])  # Extract the number after "LOS"
        tile = int(file_parts[7][1:-4])  # Extract the number after "R"

        # Assign the extracted data to the corresponding fields in the recarray
        data[i]['id'] = id
        data[i]['seed'] = seed
        data[i]['bin'] = bin
        data[i]['LOS'] = LOS
        data[i]['tile'] = tile
        # print(data[i])

    return data 

In [ ]:
# Read the file paths from master_file.txt
filename = "master_file.txt"
with open(filename, 'r') as file:
    file_paths = file.readlines()
    file_paths = [path.strip() for path in file_paths]

data = process_files(file_paths)

In [ ]:
def read_cosmo_params(file_path):
    """Reads the cosmological parameters from the .dat file.

    Parameters
    ----------
    file_path : str
        Path to the .dat file containing the cosmological parameters

    Returns
    -------
    cosmo_params : dict
        Dictionary mapping each ID to its corresponding cosmological parameters

    """

    cosmo_params = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Iterate over the lines starting from the second line
        for line in lines[1:]:
            line = line.strip()
            if line:
                parts = line.split()
                id = int(parts[0])
                params = {
                    'Om': float(parts[1]),
                    'h': float(parts[2]),
                    'w_0': float(parts[3]),
                    'sigma_8': float(parts[4]),
                    'Oc': float(parts[5])
                }
                cosmo_params[id] = params

    return cosmo_params

In [ ]:
# Path to the .dat file
dat_file_path = "/home/tersenov/shear-pipe-peaks/example/CosmoTable.dat"

# Read the cosmological parameters from the .dat file
cosmo_params = read_cosmo_params(dat_file_path)

# Map the IDs in the recarray to the corresponding cosmological parameters
mapped_params = []
for row in data:
    id = row['id']
    params = cosmo_params.get(id)
    if params:
        mapped_params.append(params)
    else:
        print(f"No parameters found for ID {id}")

# Now, 'mapped_params' will contain the corresponding cosmological parameters for each ID in the recarray

# Generate the mass map from the catalog, make the S/N maps and count the peaks

Catalog columns:

0. RA
1. DEC
2. e1_data
3. e2_data
4. w 
5. redshift_true_sim
6. gamma1_sim
7. gamma2_sim
8. kappa_sim
9. s_metacal_data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from lenspack.geometry.projections.gnom import radec2xy
from lenspack.utils import bin2d
from lenspack.image.inversion import ks93

In [ ]:
# Define the path to the catalog file
catalog_file = "/n17data/tersenov/SLICS/Cosmo_DES/16_a/LOS4/DES_MocksCat_16_a_4_Bin3_LOS4_R4.dat"

# Load the catalog data as a numpy array
catalog_data = np.loadtxt(catalog_file)
catalog_data = catalog_data.T

In [ ]:
def read_catalog_data(catalog_data):
    ra = catalog_data[0]
    dec = catalog_data[1]
    g1_sim = catalog_data[6]
    g2_sim = catalog_data[7]
    kappa_sim = catalog_data[8]
    return ra, dec, g1_sim, g2_sim, kappa_sim

In [ ]:
def create_kappa_map(ra, dec, g1_sim, g2_sim, size_x_deg=10, size_y_deg=10, pixel_size_emap_amin=0.4):
    x, y = radec2xy(np.mean(ra), np.mean(dec), ra, dec) # Project (ra,dec) -> (x,y)

    Nx = int(size_x_deg / pixel_size_emap_amin * 60)
    Ny = int(size_y_deg / pixel_size_emap_amin * 60)

    e1map, e2map = bin2d(x, y, npix=(Nx, Ny), v=(g1_sim, g2_sim)) # bin the shear field into a 2D map
    emap = np.array([e1map,e2map]) # stack the two components into a single array

    kappaE, kappaB = ks93(e1map, -e2map) # make kappa map (the minus sign has to be here for our data conventions)
    return kappaE, kappaB

In [ ]:
def add_noise_to_kappa_map(kappa_map, shape_noise, n_gal, pix_arcmin):
    sigma_noise_CFIS = shape_noise / (np.sqrt(2 * n_gal * pix_arcmin**2))
    noise_map_CFIS_z05 = sigma_noise_CFIS * np.random.randn(kappa_map.shape[0], kappa_map.shape[1]) # generate noise map
    kappa_map_noisy = kappa_map + noise_map_CFIS_z05 # Add noise to the mass map
    return kappa_map_noisy, noise_map_CFIS_z05

In [ ]:
def smooth_kappa_map(kappa_map, pixel_size_emap_amin):
    # Set the standard deviation of the Gaussian filter based on the pixel size of the kappa map
    precision_Peaks = 2 / pixel_size_emap_amin # pixel_size_emap_amin is the pixel size of the kappa map in arcminutes
    kappa_map_smoothed = ndi.gaussian_filter(kappa_map, precision_Peaks)
    return kappa_map_smoothed

In [ ]:
def convert_to_snr_map(kappa_map_smoothed, noise_map_smoothed):
    snr_map = kappa_map_smoothed / np.std(noise_map_smoothed)
    return snr_map

Make SNR map from gaussian-smoothed noisy kappa-map

In [ ]:
# Read catalog data
ra, dec, g1_sim, g2_sim, kappa_sim = read_catalog_data(catalog_data)

# Create kappa map
kappaE, kappaB = create_kappa_map(ra, dec, g1_sim, g2_sim)

# Add noise to the kappa map
n_gal = 7  # galaxy number density in gal/arcmin^2
pix_arcmin = 0.4  # pixel size in arcmin
shape_noise = 0.44  # intrinsic ellipticity dispersion
kappaE_noisy, noise_map_CFIS_z05 = add_noise_to_kappa_map(kappaE, shape_noise, n_gal, pix_arcmin)

# Smooth the noisy kappa map
kappaE_noisy_smoothed = smooth_kappa_map(kappaE_noisy, pix_arcmin)

# Compute SNR map
snr = convert_to_snr_map(kappaE_noisy_smoothed, kappaE_noisy_smoothed)


Plot:
1. Noiseless mass map
2. Noisy mass map
3. Noisy smoothed mass map
4. SNR map

In [ ]:
import matplotlib.pyplot as plt

# Plot all four maps in subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 7))

# Plot the noiseless mass map
cmap0 = axs[0, 0].imshow(kappaE, cmap='inferno', vmin=-0.01, vmax=0.01, origin='lower')
axs[0, 0].set_title("Noiseless Mass Map")
cbar0 = plt.colorbar(cmap0, ax=axs[0, 0])

# Plot the noisy mass map
cmap1 = axs[0, 1].imshow(kappaE_noisy, cmap='inferno', vmin=-0.1, vmax=0.1, origin='lower')
axs[0, 1].set_title("Noisy Mass Map")
cbar1 = plt.colorbar(cmap1, ax=axs[0, 1])

# Plot the noisy smoothed mass map
cmap2 = axs[1, 0].imshow(kappaE_noisy_smoothed, cmap='inferno', vmin=-0.1, vmax=0.1, origin='lower')
axs[1, 0].set_title("Noisy Smoothed Mass Map")
cbar2 = plt.colorbar(cmap2, ax=axs[1, 0])

# Plot the SNR map
cmap3 = axs[1, 1].imshow(snr, cmap='inferno', origin='lower')
axs[1, 1].set_title("SNR Map")
cbar3 = plt.colorbar(cmap3, ax=axs[1, 1])

# Reversing x-axis for all plots
for ax in axs.flat:
    ax.set_xlim(ax.get_xlim()[::-1])

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

Make the peak-count histogram from the SNR map and plot it

In [ ]:
import lenspack.peaks as peaks

kappa_snr = np.linspace(-2, 6, 31)  # SNR values to compute peak counts for
kappa_th_center_snr = 0.5 * (kappa_snr[:-1] + kappa_snr[1:])  # Center of SNR bins

# Compute peak counts for the SNR map
kappa_snr = np.linspace(-2, 6, 31)  # Adjust the range as needed
peak_counts_first = peaks.peaks_histogram(snr, kappa_snr)[0]

# Plot peak counts for the first SNR map
plt.plot(kappa_th_center_snr, peak_counts_first)
# plt.yscale('log')
plt.xlabel('SNR smooth')
plt.ylabel('Peak Counts')
plt.title('Peak Counts Histogram for Gaussian SNR Map')
# plt.ylim(1e-4, 1e4)
plt.show()

Make multiscale SNR maps with starlet transform

In [ ]:
from lenspack.starlet_l1norm import noise_coeff, get_l1norm_noisy
from lenspack.image.transforms import starlet2d
from astropy.stats import mad_std

In [ ]:
def compute_multiscale_snr_maps(image, noise, nscales):
    """
    Compute SNR maps for each wavelet scale of a noisy image.
    
    Parameters:
        image (numpy.ndarray): The noiseless image.
        noise (numpy.ndarray): The noise to be added to the image.
        nscales (int): Number of wavelet scales for starlet decomposition.
        
    Returns:
        snr_maps (list of numpy.ndarray): List of SNR maps for each scale.
    """
    # Add noise to the noiseless image
    image_noisy = image + noise
    
    # Perform starlet decomposition
    image_starlet = starlet2d(image_noisy, nscales)
    
    # Estimate the noise level
    noise_estimate = mad_std(image_noisy)
    coeff_j = noise_coeff(image, nscales)
    
    snr_maps = []
    for image_j, std_co in zip(image_starlet, coeff_j):
        sigma_j = std_co * noise_estimate
        
        # Compute SNR map
        snr_map = image_j / sigma_j
        snr_maps.append(snr_map)
    
    return snr_maps

In [ ]:
multiscale_snr_maps = compute_multiscale_snr_maps(kappaE, noise_map_CFIS_z05, nscales=5)

Plot the SNR maps of the 5 different scales and the coarse map

In [ ]:
# Plot SNR maps in subplots with 2 rows
fig, axs = plt.subplots(2, 3, figsize=(15, 8))

# Define vmin and vmax values for each SNR map
vmin_values = [-0.5, -3, -4, -4, -4, -4]  # Replace with your desired vmin values
vmax_values = [0.5, 3, 4, 4, 4, 4]  # Replace with your desired vmax values

scales_arcmin = [2**(i+1) * pix_arcmin for i in range(len(multiscale_snr_maps))]

for i, (snr_map, scale_arcmin) in enumerate(zip(multiscale_snr_maps, scales_arcmin)):
    row = i // 3
    col = i % 3
    # cmap = axs[row, col].imshow(snr_map, cmap='inferno', origin='lower')
    cmap = axs[row, col].imshow(snr_map, cmap='inferno', origin='lower', vmin = vmin_values[i], vmax = vmax_values[i])
    axs[row, col].set_title(f"SNR Map (Scale {scale_arcmin:.1f} arcmin)")
    cbar = plt.colorbar(cmap, ax=axs[row, col])

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


Calculate the peak counts for each scale

In [ ]:
def compute_peak_counts_on_snr_maps(snr_maps, kappa_snr):
    """
    Compute peak counts for each wavelet scale of SNR maps.

    Parameters:
        snr_maps (list of numpy.ndarray): List of SNR maps for each scale.
        kappa_snr (numpy.ndarray): Array of kappa values corresponding to SNR maps.

    Returns:
        kappa_th_center_snr (numpy.ndarray): Array of kappa threshold centers for peak counts.
        peak_counts (list of numpy.ndarray): List of peak counts for each scale.
    """
    kappa_th_center_snr = 0.5 * (kappa_snr[:-1] + kappa_snr[1:])
    
    peak_counts = [peaks.peaks_histogram(snr_map, kappa_snr)[0] for snr_map in snr_maps]

    return kappa_th_center_snr, peak_counts


Plot the peak counts for the different scales

In [ ]:
kappa_snr = np.linspace(-2, 6, 31)  # SNR values to compute peak counts for

# Compute peak counts for each scale
kappa_th_center_snr, peak_counts = compute_peak_counts_on_snr_maps(multiscale_snr_maps, kappa_snr)

# Plot peak counts for each scale
plt.figure()

for scale, peak_count in enumerate(peak_counts):
    plt.plot(kappa_th_center_snr, peak_count, label=f'Scale {scale+1}')

plt.legend()
plt.yscale('log')
plt.xticks()
plt.yticks()
plt.xlabel('SNR smooth')
plt.ylabel('Peak Counts')
plt.grid(True)
plt.xlim(-1, 6)
plt.ylim(1e-1, 1e5)
plt.title('Peak Counts for Different Scales')
plt.show()


Calculate the l1-norm and plot its histogram

In [ ]:
# Call the function to compute the L1-norm histogram
bins_l1, l1norm_histogram = get_l1norm_noisy(kappaE, noise_map_CFIS_z05, nscales=5, nbins=50)

# Plot L1-norm histograms for each scale
for scale, l1norm_hist in enumerate(l1norm_histogram):
    plt.plot(bins_l1[scale], l1norm_hist, label=f'scale {scale}')

plt.legend()
# plt.yscale('log')
plt.xticks()
plt.yticks()
plt.xlabel('L1-norm')
plt.ylabel('Frequency')
plt.grid(True)
# plt.ylim(1e-4, 1e4)
plt.xlim(0, 6)
plt.show()
